In [4]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors


def get_authenticated_service():
    # Set up credentials
    scopes = ["https://www.googleapis.com/auth/youtube.readonly"]
    client_secrets_file = r"google-api-client_secret.apps.googleusercontent.com.json"  # Replace with your client secrets file

    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes
    )
    credentials = flow.run_local_server(port=0)
    youtube = googleapiclient.discovery.build("youtube", "v3", credentials=credentials)
    return youtube

In [2]:
# Get authenticated service
youtube = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=913981478567-kv7vsikg270e76b9irdul1kesh56bbb3.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A56553%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=iV9NbnNUinlBaDLrKPqIv9qI9T0RlA&access_type=offline


In [3]:
def get_subscribed_channels(youtube):
    subscribed_channels = []

    # Get first page of subscribed channels
    request = youtube.subscriptions().list(
        part="snippet",
        mine=True,
        maxResults=50
    )

    while request is not None:
        response = request.execute()
        for item in response.get("items", []):
            channel_title = item["snippet"]["title"]
            channel_id = item["snippet"]["resourceId"]["channelId"]

            # Fetch channel statistics
            channel_data = youtube.channels().list(
                part="statistics,snippet",
                id=channel_id
            ).execute()

            video_count = channel_data["items"][0]["statistics"]["videoCount"]
            subscriber_count = channel_data["items"][0]["statistics"]["subscriberCount"]
            channel_description = channel_data["items"][0]["snippet"]["description"]

            subscribed_channels.append((channel_title, channel_id, video_count, subscriber_count, channel_description))

        request = youtube.subscriptions().list_next(request, response)

    return subscribed_channels

In [ ]:
# Get subscribed channels with additional information
subscribed_channels = get_subscribed_channels(youtube)

In [10]:
len(subscribed_channels)

442

In [19]:
import pandas as pd 
df = pd.DataFrame({'sno': range(0, len(subscribed_channels))})

In [20]:
df["Title"]=""
df["Channel ID"]=""
df["Video Count"]=""
df["Subscriber Count"]=""
df["Description"]= ""

In [24]:
# Print the list of subscribed channels with additional information
i=0
for channel_title, channel_id, video_count, subscriber_count, channel_description in subscribed_channels:
    df.loc[i,"Title"]=channel_title
    df.loc[i,"Channel ID"]= channel_id
    df.loc[i,"Video Count"]= video_count
    df.loc[i,"Subscriber Count"]= subscriber_count
    df.loc[i,"Description"] = channel_description
    i+=1

In [ ]:
df